# Mixtures of MNLs

In [6]:
import gurobipy as gp

In [7]:
import numpy as np
import numpy.random as npr
seed = 1
rng = npr.default_rng(seed)
N = 25 #25 35
M = 5 #5 10 20
v = np.zeros((N,M))
# v = rng.uniform(0.5, 1.5, (N,M))
for i in range(N):
    for j in range(M):
        utility = rng.integers(0, 2)*i*0.05 + rng.random()*0.1
        v[i, j] = max(round(utility, 2), 0.01)
theta = list(np.random.dirichlet(np.ones(M), size=1)[0])
print(v)
print(theta)

[[0.1  0.01 0.03 0.04 0.04]
 [0.1  0.08 0.05 0.13 0.03]
 [0.01 0.04 0.13 0.08 0.05]
 [0.1  0.22 0.2  0.17 0.1 ]
 [0.01 0.26 0.06 0.29 0.05]
 [0.05 0.06 0.09 0.03 0.33]
 [0.35 0.38 0.08 0.07 0.32]
 [0.43 0.01 0.09 0.44 0.4 ]
 [0.4  0.06 0.08 0.43 0.06]
 [0.08 0.47 0.5  0.04 0.51]
 [0.57 0.09 0.59 0.57 0.08]
 [0.02 0.56 0.63 0.04 0.03]
 [0.02 0.64 0.63 0.04 0.66]
 [0.04 0.07 0.69 0.71 0.73]
 [0.05 0.02 0.72 0.73 0.03]
 [0.08 0.01 0.79 0.07 0.05]
 [0.08 0.87 0.84 0.04 0.02]
 [0.03 0.03 0.06 0.08 0.93]
 [0.96 0.99 0.95 0.91 0.02]
 [0.01 1.03 0.98 1.   0.96]
 [0.04 1.03 1.01 1.07 1.04]
 [0.02 0.04 1.15 0.07 1.1 ]
 [1.14 0.09 1.2  0.08 1.16]
 [0.04 0.05 0.01 0.1  0.03]
 [1.24 1.22 0.01 1.23 0.03]]
[0.44449005431625477, 0.104624694483056, 0.18975838387569455, 0.18633658929251798, 0.07479027803247656]


In [8]:
def IP_mmnl(p):
    global v, N
    global theta, M

    #create a new model
    model = gp.Model("IP_mmnl")

    B = 10*max(p)
    # create decision variables and store them in the arrays z, x, y
    z = model.addVars(M, vtype=gp.GRB.CONTINUOUS, ub = B, name="z")
    x = model.addVars(N, M, vtype=gp.GRB.CONTINUOUS, lb=-np.infty, ub = B, name="x")
    y = model.addVars(N, vtype=gp.GRB.BINARY, name="y")
    model.update()

    # set objective function
    objExpr = gp.LinExpr()
    for j in range(M):
        objExpr += theta[j] * z[j]
    model.setObjective(objExpr, gp.GRB.MAXIMIZE)
    model.update()

    # add constraints
    for j in range(M):
        myConstr = gp.LinExpr()
        for i in range(N):
            myConstr += v[i][j] * x[i, j]
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = y[i]*B, name = "xa_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = -y[i]*B, name = "xb_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = p[i] - z[j] - (1-y[i])*B, name = "xc_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = p[i] - z[j] + (1-y[i])*B, name = "xd_" + str(i) + '_' + str(j))
        model.addConstr(lhs = z[j], sense = gp.GRB.LESS_EQUAL, rhs = myConstr, name = "z_" + str(j))
    model.update()

    # write model 
    model.write("IP_mmnl.lp")

    # solve model
    model.Params.OutputFlag = 0
    model.optimize()
    
    bestS = []
    profit = float(model.objVal)
    for i in range(N):
        if y[i].x > 0.5:
            bestS.append(i)
    bestS = sorted(bestS)
    return profit, bestS

In [9]:
def profit_mmnl(p, S):
    global v
    global theta, M
    pi = 0
    for j in range(M):
        Vj_S = 0
        num = 0
        for i in S:
            num += p[i]*v[i][j]
            Vj_S += v[i][j]
        pi += theta[j]*num/(1+Vj_S)
    return pi

In [10]:
def brute_force(p):
    global N
    # compute the profit of each assortment
    best_profit = 0
    best_S = []
    for i in range(2**N):
        binary = np.binary_repr(i, width=N)
        assortment = [int(x) for x in binary]
        S = []
        for j in range(N):
            if assortment[j] == 1:
                S.append(j)
        if best_profit < profit_mmnl(p, S):
            best_profit = profit_mmnl(p, S)
            best_S = S
    return best_profit, best_S

In [11]:
def nested_by_price_mmnl(p):
    global N
    profits = [0 for i in range(N)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i] = profit_mmnl(p, S)
    best_S = [i for i in range(np.argmax(profits)+1)]
    return max(profits), best_S

In [12]:
def find_best_option_mmnl(p, S):
    global v, N
    current_best = profit_mmnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mmnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mmnl(p, newS)
    return current_best, bestS, flag

def greedy_mmnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mmnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [13]:
import statistics as st
import time
import math

In [14]:
n_simulations = 25
avg_price = 12
seed = 1
rng = npr.default_rng(seed)
#compute prices
psim = [0 for k in range(n_simulations)]
for k in range(n_simulations):
    # psim[k] = abs(rng.normal(avg_price, avg_price, N))
    psim[k] = rng.uniform(avg_price - math.sqrt(3*avg_price), avg_price + math.sqrt(3*avg_price), N)
    # psim[k] = rng.exponential(avg_price, N)
    psim[k] = np.round(psim[k], 2)
    psim[k] = sorted(psim[k], reverse=True)

In [15]:
# problem solved using the MIP formulation
profits_IP = []
times_IP = []
bestS_IP = []

for k in range(n_simulations):

    start_time = time.time()
    profit, bestS = IP_mmnl(psim[k])
    end_time = time.time()
    if k % 10 == 0:
        print(k)

    profits_IP.append(profit)
    times_IP.append(end_time-start_time)
    bestS_IP.append(bestS)
    
avg_time_IP = st.mean(times_IP)

Using license file C:\Users\ftibe\gurobi.lic
Academic license - for non-commercial use only
0


In [ ]:
# problem solved using the nested formulation
profits_nested = []
times_nested = []
bestS_nested = []

for k in range(n_simulations):

    start_time = time.time()
    profit, bestS = nested_by_price_mmnl(psim[k])
    end_time = time.time()

    profits_nested.append(profit)
    times_nested.append(end_time-start_time)
    bestS_nested.append(bestS)
    
avg_time_nested = st.mean(times_nested)

In [ ]:
# problem solved using the greedy formulation
profits_greedy = []
times_greedy = []
bestS_greedy = []

for k in range(n_simulations):

    start_time = time.time()
    profit, bestS = greedy_mmnl(psim[k])
    end_time = time.time()

    profits_greedy.append(profit)
    times_greedy.append(end_time-start_time)
    bestS_greedy.append(bestS)
    
avg_time_greedy = st.mean(times_greedy)

In [ ]:
# counting how many instances are different
count_nested = 0
different_results_nested = []
count_greedy = 0
different_results_greedy = []

for k in range(n_simulations):
    if bestS_IP[k] != bestS_greedy[k]:
        count_greedy += 100/n_simulations
        different_results_greedy.append(100*profits_greedy[k]/profits_IP[k])
        print(k, bestS_IP[k], bestS_greedy[k], bestS_nested[k])
        
    if bestS_IP[k] != bestS_nested[k]:
        count_nested += 100/n_simulations
        different_results_nested.append(100*profits_nested[k]/profits_IP[k])
        print(k, bestS_IP[k], bestS_greedy[k], bestS_nested[k])

10 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
14 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
26 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 17] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 22] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
26 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 17] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 22] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
27 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
31 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] [0, 1, 2, 3, 4, 5, 6, 7, 8

In [ ]:
if different_results_greedy != []:
    print(' & ' + str(count_greedy) + ' & ' + str(round(min(different_results_greedy), 2)) \
          + ' & ' + str(round(st.mean(different_results_greedy),2)) \
            + ' & ' + str(round(avg_time_greedy, 2)))
else:
    print(' & 0 & NA & NA & ' + str(round(avg_time_greedy, 2)))

print('\n')

if different_results_nested != []:
    print(' & ' + str(count_nested) + ' & ' + str(round(min(different_results_nested), 2)) \
          + ' & ' + str(round(st.mean(different_results_nested),2)) \
            + ' & ' + str(round(avg_time_nested, 2)))
else:
    print(' & 0 & NA & NA & ' + str(round(avg_time_nested, 2)))

DR_g =  8.0
Min_g = 94.6905617255551
Mean_g = 97.98578754875614
t_g = 0.5173629409018269


DR_n =  4.0
Min_n = 99.89315427781176
Mean_n = 99.9694772999673
t_n = 0.05793071645052765


In [ ]:
# brute_force(psim[87])
